In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler
from utils.Dataset import Dataset
from net.networks import ResNet10
import random
import torch.random
from sklearn.metrics import precision_recall_curve, auc, roc_auc_score,average_precision_score
from sklearn.metrics import precision_score, recall_score, f1_score
import joblib
torch.manual_seed(123)
torch.cuda.manual_seed(123)
np.random.seed(123)
random.seed(123)

EHRs_DrugRel_Lab = pd.read_csv("preprocessed_data(dummy)/EHRs_DrugRel_Lab.csv")
Lab_col=EHRs_DrugRel_Lab.columns[101:136]
DrugRel_col=EHRs_DrugRel_Lab.columns[346:1741]
EHRs_DrugRel=EHRs_DrugRel_Lab.drop(Lab_col.values,axis=1)
EHRs=EHRs_DrugRel.drop(DrugRel_col,axis=1)

tc=[[1,2],[3,4],[5,6],[7,8],[9,0]]
data={"EHRs":EHRs,"EHRs_DrugRel":EHRs_DrugRel,"EHRs_DrugRel_Lab":EHRs_DrugRel_Lab}

for d in data.keys():
    data_df=data[d]
    print(d,": ")
    data_df=data_df.drop(["Sepsis_Date"],axis=1)
    
    resnet10_acc = []
    resnet10_roc = []
    resnet10_prc = []
    resnet10_pre = []
    resnet10_rec = []
    resnet10_f1  = []
    
    for tc_1, tc_2 in tc:
        print(int(tc_1/2)+1,"fold")        
        padding = pd.DataFrame(0*np.ones((len(data_df), 1764-len(data_df.columns)+1)))
        data_df = pd.concat([data_df,padding],axis=1) 
        train_data=data_df.loc[(data_df["PT_ID"]%10!=tc_1) & (data_df["PT_ID"]%10!=tc_2)]
        test_data=data_df.loc[(data_df["PT_ID"]%10==tc_1) |(data_df["PT_ID"]%10==tc_2)]
        train_feature = train_data.drop(["Label"], axis=1)
        train_label = train_data[["Label"]]
        test_feature = test_data.drop(["Label"], axis=1)
        test_label = test_data[["Label"]]        
        scaler = MinMaxScaler()
        train_feature = scaler.fit_transform(train_feature)
        test_feature = scaler.transform(test_feature)        
        rd = RandomUnderSampler()
        train_feature, train_label = rd.fit_resample(train_feature,train_label)
        train_feature=train_feature.reshape(train_feature.shape[0],1,42,42)
        test_feature=test_feature.reshape(test_feature.shape[0],1,42,42)
        
        BATCH_SIZE=int(len(train_feature)/2)        
        n_epochs=500
        LEARNING_RATE=0.00001    
        
        train_data = Dataset(torch.FloatTensor(train_feature), torch.FloatTensor(train_label.values))
        test_data = Dataset(torch.FloatTensor(test_feature), torch.FloatTensor(test_label.values))
        train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
        test_loader = DataLoader(dataset=test_data, batch_size=len(test_feature), shuffle=False)
        
        model=ResNet10()        
        model=model.cuda()
        loss_f = nn.BCELoss()
        optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, n_epochs*len(train_loader), eta_min=0)

        #forward loop
        losses = []
        accur = []
        val_losses = []
        val_accur = []        
        
        for i in range(n_epochs):
            if i == 500:
                break;
            total_loss = 0
            total_acc = 0 
            val_total_loss = 0
            val_total_acc = 0 
            for j,(x_train,y_train) in enumerate(train_loader):
                x_train,y_train=x_train.cuda(),y_train.cuda()                
                output = model(x_train)
                loss = loss_f(output,y_train.reshape(-1,1))        
                acc = (torch.round(output.reshape(-1)) == y_train.reshape(-1)).sum()/len(y_train)
        
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                scheduler.step()
                total_loss+=loss.item()
                total_acc+=acc.item()
                
            with torch.no_grad():    
                for j,(x_test,y_test) in enumerate(test_loader):
                    x_test,y_test=x_test.cuda(),y_test.cuda()
                    y_pre = model(x_test)
                    val_loss = loss_f(y_pre,y_test.reshape(-1,1))                    
                    val_acc=(torch.round(y_pre.reshape(-1)) == y_test.reshape(-1)).sum()/len(y_test)
                    val_total_loss+=val_loss.item()
                    val_total_acc+=val_acc.item()
                    
            total_loss = total_loss/len(train_loader)
            total_acc = total_acc/len(train_loader)
            val_total_loss = val_total_loss/len(test_loader)
            val_total_acc = val_total_acc/len(test_loader)
            
            if i%100 == 0:
                losses.append(loss)
                accur.append(acc)
                val_losses.append(val_loss)
                print("epoch{}\tloss:{}\tacc:{}"
                      .format(i,np.round(total_loss,3),np.round(total_acc,3)),
                      "\tval_loss:{}\tv_acc:{}"
                      .format(np.round(val_total_loss,3),np.round(val_total_acc,3)))

        y_pre=torch.round(model(torch.cuda.FloatTensor(test_feature))).cpu().detach().numpy()
        y_proba=model(torch.cuda.FloatTensor(test_feature)).cpu().detach().numpy()
        
        y_label = test_label.values
        
        test_acc  = (y_pre==y_label).sum()/len(y_label)
        AUROC     = roc_auc_score(y_label, y_proba)
        AUPRC     = average_precision_score(y_label, y_proba)
        precision = precision_score(y_label, y_pre, pos_label=1)
        recall    = recall_score(y_label, y_pre)
        f1_score_ = f1_score(y_label, y_pre)
        
        resnet10_acc = resnet10_acc + [test_acc]
        resnet10_roc = resnet10_roc + [AUROC]
        resnet10_prc = resnet10_prc + [AUPRC]
        resnet10_pre = resnet10_pre + [precision]
        resnet10_rec = resnet10_rec + [recall]
        resnet10_f1  = resnet10_f1  + [f1_score_]

        #torch.save(model.state_dict(), "trained_model/"+d+"_"+str(tc_1)+"_resnet10.pt")
        #joblib.dump(scaler, "trained_model/"+d+"_"+str(tc_1)+"_scaler_resnet10.pkl")        
    
    print("\n")
    print("ResNet10 : ")
    print("  Acc      : ",np.round(np.array(resnet10_acc).mean(),3))
    print("  AUROC    : ",np.round(np.array(resnet10_roc).mean(),3))
    print("  AUPRC    : ",np.round(np.array(resnet10_prc).mean(),3))
    print("  Precision: ",np.round(np.array(resnet10_pre).mean(),3))
    print("  Recall   : ",np.round(np.array(resnet10_rec).mean(),3))
    print("  F1-score : ",np.round(np.array(resnet10_f1 ).mean(),3))
    print("\n")

EHRs : 
1 fold
epoch0	loss:0.773	acc:0.5 	val_loss:0.633	v_acc:0.69
epoch100	loss:0.572	acc:0.741 	val_loss:0.615	v_acc:0.679
epoch200	loss:0.45	acc:0.815 	val_loss:0.589	v_acc:0.701
epoch300	loss:0.358	acc:0.883 	val_loss:0.587	v_acc:0.716
epoch400	loss:0.321	acc:0.904 	val_loss:0.591	v_acc:0.72
2 fold
epoch0	loss:0.689	acc:0.538 	val_loss:0.699	v_acc:0.391
epoch100	loss:0.551	acc:0.744 	val_loss:0.662	v_acc:0.619
epoch200	loss:0.401	acc:0.858 	val_loss:0.657	v_acc:0.669
epoch300	loss:0.299	acc:0.916 	val_loss:0.658	v_acc:0.672
epoch400	loss:0.263	acc:0.93 	val_loss:0.658	v_acc:0.692
3 fold
epoch0	loss:0.707	acc:0.496 	val_loss:0.739	v_acc:0.331
epoch100	loss:0.552	acc:0.753 	val_loss:0.647	v_acc:0.62
epoch200	loss:0.412	acc:0.843 	val_loss:0.656	v_acc:0.613
epoch300	loss:0.31	acc:0.911 	val_loss:0.688	v_acc:0.631
epoch400	loss:0.272	acc:0.933 	val_loss:0.708	v_acc:0.638
4 fold
epoch0	loss:0.689	acc:0.54 	val_loss:0.699	v_acc:0.556
epoch100	loss:0.537	acc:0.753 	val_loss:0.703	v_acc:0